# Search Engine

In this file, we walk through the process of creating a search engine with an inverted index.

In [24]:
# Imports
import pandas as pd
from path import Path
import os
from collections import defaultdict
from math import log
import string
from collections import Counter
import re

In [25]:
# Start by loading in the data
complete = pd.read_csv('https://scmcqueen.github.io/StarTrekScriptData/complete_data.csv')
# Rename columns
complete.columns = ['index', 'character', 'quote', 'scene', 'location', 'view',
       'episode', 'date', 'series', 'file']
# Clean up Character & Quote
complete['character'] = complete['character'].apply(lambda text: " ".join(str(text).split()))
complete['quote']=complete['quote'].apply(lambda text: " ".join(text.split()))
# Show sample of data
complete.sample(5)

,index,character,quote,scene,location,view,episode,date,series,file
141466,198,WORF,Unidentified craft Sector four to Sector four ...,35 INT. MAIN BRIDGE,MAIN BRIDGE,INT.,The Outrageous Okona,1988-10-04,The Next Generation,130.txt
441,441,ROM,I wasn't working.,70 INT. QUARK'S,QUARK'S,INT.,STAR TREK: DEEP SPACE NINE,1996-08-29,Deep Space Nine,504.txt
75689,434,WORF,Very well. You may cross the border. But only ...,57 INT. MEDICAL SHIP - BRIDGE - FUTURE (OPTI...,MEDICAL SHIP - BRIDGE - FUTURE,INT.,All Good Things...,1994-03-10,The Next Generation,277.txt
66089,231,QUARK,Account number CJ57436.,22 INT. SECURITY OFFICE,SECURITY OFFICE,INT.,Who Mourns for Morn?,1997-10-28,Deep Space Nine,536.txt
55570,284,ISHKA,He said I didn't really love him... that I was...,25 EXT. ISHKA'S HOUSE,ISHKA'S HOUSE,EXT.,Ferengi Love Songs,1997-01-30,Deep Space Nine,518.txt


In [26]:
# TO DO: POTENTIALLY FIX CHARACTER NAME TOO

Now that we have the data loaded in, we may want to define some functions that we will use when creating the search engine.

In [27]:
def normalize_string(input_string: str) -> str:
    '''This function processes a string by removing punctuation,
    making text lowercase, and getting rid of extra spaces

    For example:
        "Hello,  HI!!! How are     you?"
    becomes
        "hello hi how are you"
    '''
    translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    string_without_punc = input_string.translate(translation_table)
    string_without_double_spaces = ' '.join(string_without_punc.split())
    return string_without_double_spaces.lower()

In [28]:
# MAY WANT TO ALSO LEMMATIZE & DROP STOP WORDS

In [29]:
def update_url_scores(old: dict[str, float], new: dict[str, float]):
    '''This function adds two dictionaries together'''
    for url, score in new.items():
        if url in old:
            old[url] += score
        else:
            old[url] = score
    return old

Now we will create a Search Engine object with an inverted index.

In [ ]:
class search_engine:
    '''This class creates a search engine object'''
    def __init__(self, index:dict[str, dict[str, int]]=None, docs: dict[str, str]=None,
        original_docs: dict[str, str]=None,k1:float=1.5,b:float=0.75,
        name:str='Default Search Engine',full_data: pd.DataFrame=None):
        '''
        Instantiate an instance of the search engine class.

        Input:
            index: dict[str, dict[int,int]], the inverted index
            docs: dict[int, str], key is the id of the quote and value is the quote text
            original_docs:
            k1: float, k1 constant to use for bm25
            b: float, b constant to use for bm25
            name: string, name used for the search engine instance
            full_data:  pd.DataFrame, the original dataset used

        Output:
            search engine!
        '''
        # set index
        if index is None: self.index = defaultdict(lambda: defaultdict(int))
        else: self.index = index
        # set docs
        if docs is None: self.docs = {}
        else: self.docs = docs
        # set original docs
        if original_docs is None: self.original_docs = {}
        else: self.original_docs = original_docs
        # set k1
        self.k1 = k1
        # set b
        self.b = b
        # set name
        self.name = name
        # set full_data
        self.full_data = full_data

    def __str__(self)->str:
        '''
        Prints a readable name of the search engine

        Output:
            str: name of the instance
        '''
        return(self.name)

    def bulk_load(self,data:dict,full_data:pd.DataFrame=None)->None:
        '''
        Bulk loads new documents to the search engine.

        Input:
            data: dict, the formatted data
            full_data: pd.DataFrame, the data with the full info
        '''
        # get the original size of the docs
        original_len = len(self.docs.keys())
        # for each index in the data
        for ind in data.keys():
            content = data[ind] # quote text
            # add to original docs
            self.original_docs[ind]=content
            # normalize content & add to docs
            n_content = normalize_string(str(content))
            self.docs[ind]=n_content

            # now we want to created the inverted index based on words
            words = n_content.split(" ")
            for w in words:
                self.index[w][ind]+=1 # update count of word per index
        # get new length
        new_len = len(self.docs.keys())
        print(f'We added {new_len-original_len} documents. The engine now has {new_len} documents.')

    def individual_load(self, document:str)-> None:
        '''
        Load a single document into the search engine. Ideally this should not be used.

        Input:
            document: str, the new text document to add to the search engine.
        '''
        # assign new id
        new_id = len(self.docs.keys())
        # add to docs & original docs
        self.original_docs[new_id]=document
        n_docs = normalize_string(document)
        self.docs[new_id]=n_docs
        # now we need to update the inverted index
        words = n_docs.split(" ")
        for w in words:
            self.index[w][new_id]
        print(f'Added document "{document}" to search engine.')

    def num_docs(self)->int:
        '''
        Returns the number of docs

        Output:
            int: length of docs
        '''
        return len(self.docs.keys())

    def find_ids(self, keyword:str)->dict:
        '''
        Find the doc ids that contain a keyword.

        Input:
            keyword: str, the word to search
        Returns:
            dict: keys are the indices and the values are
                the frequency of the word in the document
        '''
        key = normalize_string(keyword)
        return(self.index[key])

    def bw_idf(self,keyword:str)-> float:
        '''
        Find the inverse document frequency for a term

        Input:
            keyword: str, word to search

        Output:
            float: the idf score
        '''
        num_docs = self.num_docs()
        keyword = normalize_string(keyword)
        n_kw = len(self.find_ids(keyword))
        idf = log((num_docs-n_kw+0.5)/(n_kw+0.5)+1)
        return(idf)

    def bm25(self,keyword:str)-> dict[str, float]:
        '''
        Calculate the bm25 score for every document

        Input:
            keyword: str, word to search

        Output:
            dict[str, float]: dict of doc ids & the bm25 score
        '''
        result = {} # instantiate the output
        keyword = normalize_string(keyword)
        idf = self.bw_idf(keyword) # get the idf score
        # get the avg len of a document
        avg_ql = sum(len(d) for d in self.docs.values()) / len(self.docs)
        # calculate the bw score for each
        for id, freq in self.find_ids(keyword).items(): # for doc id & word freq
            numerator = freq*(self.k1+1)
            denominator = freq+self.k1*(1 - self.b + self.b * len(self.docs[id]) / avg_ql)
            result[id]=idf*numerator / denominator
        # return dict with the ids & scores
        return result

    def bw_search(self,query:str,limit:int=None,context:bool=False)->dict[str,float]:
        '''
        Completes the bm25 search of the documents using the query and returns

        Input:
            query: str, the query to search through the documents
            limit: int, limits the number of documents
            context: bool, if true return also the bm-scores of the lines before & after the selected lines.
                There must exist a limit for this to be true.

        Output:
            dict[str,float]: the index and the bm25 score
        '''
        # split the query & normalize it
        kws = normalize_string(query).split(" ")
        scores = {} # initialize output
        for k in kws:
            kw_score = self.bm25(k) # get the scores for this word
            scores = update_url_scores(scores,kw_score) # add the dict values together
        # sort the scores by the bm25 score
        sorted_scores = sorted(scores.items(), key=lambda kv: (kv[1], kv[0]),reverse=True)
        output = sorted_scores.copy()
        # limit the score output
        if limit is not None:
            output = sorted_scores[:limit]
            # check if we need to get context
            if context:
                # get the top x indices
                top_ind = [x[0] for x in output]
                # get the context indices (lines before & after)
                ctx_ind = [y-1 for y in top_ind]+[y+1 for y in top_ind]
                # get their bm_25
                more_bm25 = [(z, scores[z]) if z in scores.keys() else (z,0) for z in ctx_ind]
                # add to output
                output = output+more_bm25
        return(output)


In [ ]:
# SHOULD I ADD STRUCTURED BM25

In [53]:
# create instance of search engine
bm25_engine = search_engine(name='BM25 Engine',full_data=complete)
# load data in bulk
bm25_engine.bulk_load(complete[['quote']].to_dict()['quote'])

We added 144211 documents. The engine now has 144211 documents.


## Now I have a search engine! Let's do a test query

In [54]:
q_results = bm25_engine.bw_search('dabo',20,context=True)

In [55]:
q_results

[(38743, 14.950076796302916),
 (12663, 12.64571859691301),
 (61458, 12.50535557934936),
 (61353, 12.50535557934936),
 (58731, 12.50535557934936),
 (53295, 12.50535557934936),
 (46327, 12.50535557934936),
 (46320, 12.50535557934936),
 (38752, 12.50535557934936),
 (33055, 12.50535557934936),
 (32506, 12.50535557934936),
 (32500, 12.50535557934936),
 (25198, 12.50535557934936),
 (25187, 12.50535557934936),
 (23268, 12.50535557934936),
 (17130, 12.50535557934936),
 (11519, 12.50535557934936),
 (1980, 12.50535557934936),
 (23269, 11.785538827271429),
 (11518, 11.59258591664298),
 (38742, 0),
 (12662, 0),
 (61457, 0),
 (61352, 0),
 (58730, 0),
 (53294, 5.201571163523684),
 (46326, 0),
 (46319, 0),
 (38751, 0),
 (33054, 0),
 (32505, 0),
 (32499, 0),
 (25197, 0),
 (25186, 0),
 (23267, 0),
 (17129, 0),
 (11518, 11.59258591664298),
 (1979, 0),
 (23268, 12.50535557934936),
 (11517, 0),
 (38744, 0),
 (12664, 0),
 (61459, 0),
 (61354, 0),
 (58732, 0),
 (53296, 0),
 (46328, 0),
 (46321, 0),
 (38753,